### Imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_seq_items', None)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
np.set_printoptions(threshold=np.nan)

import warnings

warnings.filterwarnings('ignore')
import argparse

import sys, os

sys.path.append(os.getcwd())

import os
import requests
from requests.exceptions import ConnectionError

import bs4
from bs4 import BeautifulSoup
from fastnumbers import isfloat
from fastnumbers import fast_float
from multiprocessing.dummy import Pool as ThreadPool 
import more_itertools
from random import shuffle





### Libs

In [9]:
def ffloat(string):
    if string is None:
        return np.nan
    return fast_float(string.split(" ")[0].replace(',',''),default=np.nan)

def get_children(html_content):
    return [item for item in html_content.children if type(item)==bs4.element.Tag]

def get_portfolio(mfid):
    url = "https://www.moneycontrol.com/india/mutualfunds/mfinfo/portfolio_holdings/"+mfid
    page_response = requests.get(url, timeout=10)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    portfolio_table = page_content.find('table', attrs={'class': 'tblporhd'})
    fund_name = page_content.find('h1').text
    return portfolio_table,fund_name

def get_table(portfolio_table):
    portfolio_elems = get_children(portfolio_table)

    table_data = list()
    for row in portfolio_elems:
        row_data = list()
        row_elems = get_children(row)
        for elem in row_elems:
            text = elem.text.strip().replace("\n","")
            if len(text)==0:
                continue
            elem_descriptor = {'text':text}
            elem_children = get_children(elem)
            if len(elem_children)>0:
                if elem_children[0].has_attr('href'):
                    elem_href = elem_children[0]['href']
                    elem_descriptor['href'] = elem_href

            row_data.append(elem_descriptor)
        table_data.append(row_data)
    return table_data

In [38]:

def get_scrip_info(url):
    if not url.startswith("http"):
        url = "https://www.moneycontrol.com"+url
    try:
        page_response = requests.get(url, timeout=10)
    except Exception as e:
        return None
    
    
    page_content = BeautifulSoup(page_response.content, "html.parser")
    try:
        price = page_content.find('div',attrs={'id':'Nse_Prc_tick_div'}).text.split(" ")[0].replace(',','')
        name = page_content.find('h1',attrs={'class':'company_name'}).text
        html_data_content = page_content.find('div', attrs={'id': 'mktdet_1'})
        petable = get_table(get_children(html_data_content)[0])
        pbtable = get_table(get_children(html_data_content)[1])
        data_table = list()
        data_table.extend(petable)
        data_table.extend(pbtable)

        consolidated_html_data_content = page_content.find('div', attrs={'id': 'mktdet_2'})
        consolidated_petable = get_table(get_children(consolidated_html_data_content)[0])
        consolidated_pbtable = get_table(get_children(consolidated_html_data_content)[1])
        consolidated_data_table = list()
        consolidated_data_table.extend(consolidated_petable)
        consolidated_data_table.extend(consolidated_pbtable)
    except Exception as e:
        #print(url)
        #print(e)
        return None
    
    key_val_pairs = {}
    
    for row in consolidated_data_table:
        
        k = row[0]['text']
        if len(row)<2:
            v=None
        else:
            v = row[1]['text'].split(" ")[0].replace(',','')
        key_val_pairs[k]=v
            
    for row in data_table:
        
        k = row[0]['text']
        if len(row)<2:
            v=None
        else:
            v = row[1]['text'].split(" ")[0].replace(',','')
        
        if k not in key_val_pairs or not isfloat(key_val_pairs[k]):
            key_val_pairs[k]=v
        
    try:    
        key_val_pairs["pe"] = key_val_pairs.pop('P/E')
        key_val_pairs["book_value"] = key_val_pairs.pop('BOOK VALUE (Rs)')
        key_val_pairs["deliverables"] = key_val_pairs.pop('DELIVERABLES (%)')
        key_val_pairs["eps"] = key_val_pairs.pop('EPS (TTM)')
        key_val_pairs["industry_pe"] = key_val_pairs.pop('INDUSTRY P/E')
        key_val_pairs["market_cap"] = key_val_pairs.pop('MARKET CAP (Rs Cr)')
        key_val_pairs["pb"] = key_val_pairs.pop('PRICE/BOOK')
        key_val_pairs["pc"] = key_val_pairs.pop('P/C')
        key_val_pairs['price'] = price
        key_val_pairs["name"] = name
        del key_val_pairs['DIV (%)']
        del key_val_pairs['DIV YIELD.(%)']
        del key_val_pairs['FACE VALUE (Rs)']
        del key_val_pairs['Market Lot']
    except Exception as e:
        print(url)
        print(e)
        return None
    return key_val_pairs




def get_all_details(scrip_links_table,percent_col=4,scrip_col=0):
    scrip_details = list()
    percent_col = 4
    scrip_col = 0
    qty_col = 2
    total_value_crores_col = 3
    
    def scrip_detail_collector(row):
        scrip_url = row[scrip_col]['href']
        scrip_detail = get_scrip_info(scrip_url)
        scrip_detail['percent'] = row[percent_col]['text']
        scrip_detail['name'] = row[scrip_col]['text']
        scrip_detail['qty'] = row[qty_col]['text']
        scrip_detail['total_value_crores'] = row[total_value_crores_col]['text']
        return scrip_detail
        
    pool = ThreadPool(8)
    scrip_details = pool.map(scrip_detail_collector, scrip_links_table)
    scrip_details = list(filter(lambda x:x is not None),scrip_details)
    scrip_details = pd.DataFrame.from_records(scrip_details)
    numeric_cols = ['book_value', 'price','deliverables', 'eps', 'industry_pe', 'market_cap', 'pb', 'pc', 'pe', 'percent','qty','total_value_crores']
    scrip_details[numeric_cols] = scrip_details[numeric_cols].applymap(ffloat)
    return scrip_details

### MF Analysis

In [14]:
# get avg pe and pb and deliverables
# get avg market cap
# current nav
# current aum
# last five year growth rate
# mfid = "MSB532"
# nifty MKM321 MSB1174
# Bluechip SBI MSB532

In [16]:
def comparative_analysis(fund_list):
    fund_details = list()
    for fund in fund_list:
        portfolio_table,fund_name = get_portfolio(mfid = fund)
        table_data = get_table(portfolio_table)
        scrip_details = get_all_details(table_data[1:])
        pe = np.dot(scrip_details['price'].fillna(0),scrip_details['percent'])/np.dot(scrip_details['eps'].fillna(0),scrip_details['percent'])
        pb = np.dot(scrip_details['price'].fillna(0),scrip_details['percent'])/np.dot(scrip_details['book_value'].fillna(0),scrip_details['percent'])
        aum = np.sum(scrip_details['total_value_crores'])
        avg_market_cap = np.dot(scrip_details['market_cap'].fillna(0),scrip_details['percent']/100)
        
        fund_detail = {"name":fund_name,"pe":pe,"pb":pb,"aum":aum,"avg_market_cap":avg_market_cap}
        fund_details.append(fund_detail)
    return pd.DataFrame.from_records(fund_details)

In [17]:
comparative_analysis(['MSB532','MKM321'])

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


,aum,avg_market_cap,name,pb,pe
0,4452.41,159431.300454,SBI Blue Chip Fund - Direct Plan (G),5.627699,34.015525
1,500.89,289569.418363,Kotak Nifty ETF,4.590008,28.354778


### Company Analysis and Filtering 



#### Criterias - Current
- both NSE and BSE listing
- Not PSU
- Market Cap >= 200 Cr
- Low P/E, pe<20 in all cases, pe wrt to MarketCap and eps increase
- Low debt/Equity Ratio, de<1 in all cases
- P/B ratio

#### Criterias - Over time 
- Increasing Sale YoY
- Decreasing d/e ratio YoY
- Increasing eps YoY
- No Steep Price increase in last 2 years

#### Criterias - Buy Now
- Below 30,100 DMA
- Low Volumes 

#### Find Good Companies from MF
- Use above criterias

#### Testing for P/E and marketcap

In [229]:
def pe_comparator_test(mcap,pe):
    mpow = 0.47
    pepow = 1.59
    thres = 0.8
    if mcap<1e3:
        pepow = 1.63
        mpow = 0.42
        thres = 1.0
    score = (np.power(mcap,mpow)/(np.power(pe,pepow)))
    return score

In [230]:
pe_list = list()
pe_set = set()
for pe in [1,2,3,4,5,6,7,8,9,10]:
    for mcap in np.arange(1e2,2e3,1e2):
        if pe not in pe_set:
            if pe_comparator_test(mcap,pe)>1.0:
                pe_list.append({"mcap":mcap,"pe":pe,"result":pe_comparator_test(mcap,pe)})
                pe_set.add(pe)
pd.DataFrame.from_records(pe_list)

,mcap,pe,result
0,100.0,1,6.918310
1,100.0,2,2.235224
2,100.0,3,1.154230
3,300.0,4,1.145600
4,600.0,5,1.065375
5,1000.0,6,1.488468
6,1000.0,7,1.164915
7,1200.0,8,1.026366
8,1700.0,9,1.002456


In [231]:
pe_list = list()
pe_set = set()
for pe in [1,10,11,13,15,17,18,19,20,21,22,25,30]:
    for mcap in np.arange(1e3,2e4,1e3):
        if pe not in pe_set:
            if pe_comparator_test(mcap,pe)>0.8:
                pe_list.append({"mcap":mcap,"pe":pe,"result":pe_comparator(mcap,pe)})
                pe_set.add(pe)
pd.DataFrame.from_records(pe_list)


,mcap,pe,result
0,1000.0,1,25.703958
1,2000.0,10,0.915133
2,3000.0,11,0.951552
3,4000.0,13,0.835215
4,6000.0,15,0.804908
5,10000.0,17,0.838661
6,11000.0,18,0.800887
7,14000.0,19,0.823118
8,16000.0,20,0.807790
9,19000.0,21,0.810374


#### Actual P/E function

In [24]:
def pe_comparator_filter(stock_detail):
    pe = ffloat(stock_detail['pe'])
    mcap = ffloat(stock_detail['market_cap'])
    mpow = 0.47
    pepow = 1.59
    thres = 0.8
    if mcap<1e2:
        return False
    if mcap<1e3:
        pepow = 1.63
        mpow = 0.42
        thres = 1.0
    score = (np.power(mcap,mpow)/(np.power(pe,pepow)))
    return score>thres

#### Getting All Company Links from MoneyControl

In [6]:


def get_stock_urls_from_listing_page(listing_page):
    page_response = requests.get(listing_page, timeout=10)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    urls_table = page_content.find('table',attrs={'class':'pcq_tbl'})
    links = list(map(lambda x:get_children(x)[0]['href'],urls_table.find_all('td')))
    return links

def get_all_links():
    abc = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    letters = [letter for letter in abc]
    listing_page_urls = ['https://www.moneycontrol.com/india/stockpricequote/'+letter for letter in letters]
    pool = ThreadPool(8)
    all_links = list(more_itertools.flatten(pool.map(get_stock_urls_from_listing_page, listing_page_urls)))
    return all_links

In [35]:
def get_all_company_details(filters=[],size=200):
    # filters is a list of functions returning T/F, They are always
    all_links = get_all_links()
    shuffle(all_links)
    all_links = all_links[:size]
    pool = ThreadPool(8)
    scrip_details = pool.map(get_scrip_info, all_links)
    scrip_details = list(filter(lambda x:x is not None,scrip_details))
    for filtr in filters:
        scrip_details = filter(filtr,scrip_details)
    scrip_details = pd.DataFrame.from_records(scrip_details)
    numeric_cols = ['book_value', 'price','deliverables', 'eps', 'industry_pe', 'market_cap', 'pb', 'pc', 'pe',]
    scrip_details[numeric_cols] = scrip_details[numeric_cols].applymap(ffloat)
    return scrip_details
    
    

In [33]:
all_company_df = get_all_company_details(filters=[pe_comparator_filter])
all_company_df

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

In [37]:
all_company_df = get_all_company_details(filters=[pe_comparator_filter],size=9000)
all_company_df

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


KeyError: 'MARKET CAP (Rs Cr)'